In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from math import sqrt, e, pi
from scipy.stats import norm
from datetime import datetime, timedelta
import pandas_datareader.data as pdr

# **Black Scholes Merton Model**

The Black-Scholes-Merton Model is a mathematical framework used to estimate the price of European-style options - contracts that can only be exercised on their expiration date, unlike American options, which allow execution at any time before expiration.

<br/>

<h1 style="font-size:20px"> <b>  Assumptions of the Model: </b> </h1>

- No dividends are paid out during the life of the option.
- Market returns follow a random walk.
- There are no transaction costs in buying the option.
- The risk-free rate and volatility of the underlying asset are known and constant.
- The returns on the underlying asset are log-normally distributed.
- The option is European and can only be exercised at expiration.

<br/>

Once we take these assumptions for granted, the model provides the following formulas to compute the options:

<br/> 

<div style="text-align: center; font-size:17px">  

*European Call Option Value* $= S_0 N(d_1) - Ke^{-rT} N(d_2)$

*European Put Option Value* $=  Ke^{-rT} N(-d_2) - S_0 N(-d_1)$

</div>
</br>

The terms *N(d1)* is the cumulative probability that a random draw from a standard normal distribution will be less or equal to d1, and represents the probability that the option will finish in the money, adjusted for the fact that the payoff is based on the current asset price. </br>
Similarly, *N(d2)* is the cumulative probability that a random draw from a standard normal distribution will be less than or equal to d2, and represents the probability that the option will be exercised (the option is in the money at expiration).


</br>

<div style="text-align: center; font-size:16px">  

$\Large{d_1 = \frac{ln(\frac{S_0}{K})+(r  + \frac{\sigma^2}{2})T}{\sigma\sqrt{T}}}$

$\Large{d_2 = \frac{ln(\frac{S_0}{K})+(r  - \frac{\sigma^2}{2})T}{\sigma\sqrt{T}}}$

</div>

Where:
- ${S_0}=$ Underlying Asset Price
- ${K=}$ Option Strike Price
- ${r=}$ Interest Rate
- $\sigma=$ volatility
- $T=$ Time

Let's try this with an example: The current price of a stock is 42$, its strike price is 40$, the risk-free rate is 10%, the volatility is 20%, and the strike date is in 6


In [2]:
S = 42
K = 40
T = 6
t = T/12
r = 0.1
std = 0.2

d1 = (np.log(S/K) + (r + 0.5 * std**2) * t) / (std * sqrt(t))
d2 = (np.log(S/K) + (r - 0.5 * std**2) * t) / (std * sqrt(t))

p = K * e**(-r*t) * norm.cdf(-d2) - S * norm.cdf(-d1)
c = S * norm.cdf(d1) - K * e**(-r * t) * norm.cdf(d2)

print('Put price: {:.3f} \nCall price: {:.3f}'.format(p, c))


Put price: 0.809 
Call price: 4.759


To better implement the method, I create a class *option*, which allows to keep the code organized, and it makes it more simple to add methods, as we will do later.

In [3]:

class option:

    # defining the basis to compute the option price
    def __init__(self, S, K, today, T, vol, r):
        # instead of putting a range time, we set two dates: today and the strike date, 
        # which can easily be specified with datetime
        self.t = (T - today).days/365
        self.S = S
        self.K = K
        self.r = r
        self.vol = vol

    # compute d1
    def d1(self):
        d1 = (np.log(self.S/self.K) + (self.r + 0.5 * self.vol**2) * self.t) /  \
            (self.vol * sqrt(self.t))
        return d1
    
    # compute d1
    def d2(self):
        d2 = (np.log(self.S/self.K) + (self.r - 0.5 * self.vol**2) * self. t) /  \
            (self.vol * sqrt(self.t))
        return d2
    
    # compute price
    def price(self):
        d1 = self.d1()
        d2 = self.d2()

        put = self.K * e**(-self.r * self.t) * norm.cdf(-d2) - self.S * norm.cdf(-d1)
        call = self.S * norm.cdf(d1) - self.K * e**(-self.r * self.t) * norm.cdf(d2)
        
        return put, call


# I report the same example above
S = 42
K = 40
T = 6
r = 0.1
std = 0.2

option_example = option(S = S, K = K, r = r, 
                      today = datetime(2020, 1, 1).date(), T = datetime(2020, 7, 1).date(), 
                      vol = std)

put, call = option_example.price()

print('Put price: {:.3f} \nCall price: {:.3f}'.format(put, call))

# The difference from the above value is given by the fact that here we use actual dates, instead of a specific number

Put price: 0.808 
Call price: 4.753


# **Black Scholes Merton Model addition: dividend yield**

The assumptions of the model are quite restricted, but the importance of the framework is not in discussion. However, we can clearly adapt it by adding some basic things, starting with the dividend yield $q$. </br>
Therefore, the formula for *d1* and *d2* are as follows:

<center> 

$\Large{d_1 = \frac{ln(\frac{S_0}{K})+(r - q + \frac{\sigma^2}{2})T}{\sigma\sqrt{T}}}$

$\Large{d_2 = \frac{ln(\frac{S_0}{K})+(r - q - \frac{\sigma^2}{2})T}{\sigma\sqrt{T}}}$

</center>


In [4]:
class option_2:
    
    # defining the basis to compute the option price
    def __init__(self, S, K, today, T, vol, r, dividend = 0.0):
        self.t = (T - today).days/365
        self.S = S
        self.K = K
        self.r = r
        self.vol_put = vol[0]
        self.vol_call = vol[1]
        self.dividend = dividend

    # compute d1 for the put option
    def d1_put(self):
        d1_put = (np.log(self.S/self.K) + (self.r - self.dividend + 0.5 * self.vol_put**2) * self.t) /  \
            (self.vol_put * sqrt(self.t))
        return d1_put
    
    # compute d2 for the put option
    def d2_put(self):
        d2_put = (np.log(self.S/self.K) + (self.r - self.dividend - 0.5 * self.vol_put**2) * self. t) /  \
            (self.vol_put * sqrt(self.t))
        return d2_put
    
    # compute d1 for the call option
    def d1_call(self):
        d1_call = (np.log(self.S/self.K) + (self.r - self.dividend + 0.5 * self.vol_call**2) * self.t) /  \
            (self.vol_call * sqrt(self.t))
        return d1_call

    # compute d2 for the call option
    def d2_call(self):
        d2_call = (np.log(self.S/self.K) + (self.r - self.dividend - 0.5 * self.vol_call**2) * self. t) /  \
            (self.vol_call * sqrt(self.t))
        return d2_call
    
    # compute price
    def price(self):
        d1_put = self.d1_put()
        d2_put = self.d2_put()
        d1_call = self.d1_call()
        d2_call = self.d2_call()

        put = self.K * e**(-self.r * self.t) * norm.cdf(-d2_put) - self.S * e**(-self.dividend * self.t) * norm.cdf(-d1_put)
        call = self.S * e**(-self.dividend * self.t) * norm.cdf(d1_put) - self.K * e**(-self.r * self.t) * norm.cdf(d2_put)
        
        return put, call
    

# I report the same example above, and I do not specify a dividend, as by default it is 0
S = 42
K = 40
r = 0.1
std = 0.2

option_example = option_2(S = S, K = K, r = r, 
                      today = datetime(2020, 1, 1).date(), T = datetime(2020,7, 1).date(), 
                      vol = [std, std], dividend = 0)

put, call = option_example.price()

print('Put price: {:.3f} \nCall price: {:.3f}'.format(put, call))

Put price: 0.808 
Call price: 4.753


# Algorithm in Practice
In practice, extracting by hand all the data we need to make the computation is time consuming. Fortunately, it is possible to apply some methods to get automatically all the information I need. As an example, I set up a pipeline using the Google stock (this will work only during trading days):

In [5]:
# set the spot date: the algorithm does not work if it is not a trading day
spot_date = datetime.today().date()

# set the initial strike date: a further functin will look for the right date of the option
strike_date = (datetime.today() + timedelta(30)).date()

# create a variable with the info of the stock
tk = yf.Ticker("GOOG")

# extract some information from the ticker
tk_info = [tk.info.get(dividend) for dividend in ['dividendYield', 'bid', 'ask']]
div_yield = tk_info[0]
spot_price = (tk_info[1] + tk_info[2])/2

# create an hypotetical strike price and round it to multiples of 5
strike_price = spot_price * 1.025
strike_price = np.floor(strike_price/5) * 5 



print('Spot_date:', spot_date)
print('Strike date:', strike_date)
print('Dividend yield:', div_yield)
print('Spot price:', np.round(spot_price, 3))
print('Strike price:', strike_price)

Spot_date: 2024-09-26
Strike date: 2024-10-26
Dividend yield: 0.0049
Spot price: 164.005
Strike price: 165.0


In [6]:
# checl for the different dates of options, and change the strike date into the nearest one
dates_list = [datetime.strptime(date, '%Y-%m-%d').date() for date in tk.options]
strike_date = min(dates_list, key=lambda date: abs(date - strike_date))


# extract the implied volatility of the option for the specific strike price. 
# The implied volatility reflects the market's expectations of the future volatility
# of the underlying asset's price over the life of the option.
mask_put = tk.option_chain( str(strike_date) ).puts['strike'] == strike_price
mask_call = tk.option_chain( str(strike_date) ).calls['strike'] == strike_price

put_vol = tk.option_chain( str(strike_date) ).puts[mask_put]['impliedVolatility']
call_vol = tk.option_chain( str(strike_date) ).calls[mask_call]['impliedVolatility']


# set the vol input for the function
vol = [float(put_vol), float(call_vol)]



print('Actual strike date:', strike_date)
print("Nearest date from list : " + str(strike_date))
print('Vol:', vol)


Actual strike date: 2024-10-25
Nearest date from list : 2024-10-25
Vol: [0.2804637384033203, 0.30811238769531246]


In [7]:
# extract the interest rate of a 1 month Treasury Securities from the FRED website
risk_free = pdr.get_data_fred('DGS1MO', start = spot_date - timedelta(15), end = spot_date).iloc[-1, 0]/100

print('Risk free:', np.round(risk_free, 4))


Risk free: 0.0478


In [8]:
# print all the information
print('Strike price:', strike_price)
print('Spot price:', np.round(spot_price, 3))
print('Risk free:', np.round(risk_free, 4))
print('Spot_date:', spot_date)
print('Strike date:', strike_date)
print('Dividend yield:', div_yield)
print('Vol:', vol)

Strike price: 165.0
Spot price: 164.005
Risk free: 0.0478
Spot_date: 2024-09-26
Strike date: 2024-10-25
Dividend yield: 0.0049
Vol: [0.2804637384033203, 0.30811238769531246]


In [9]:
# call the formula with the information gathered above
google_example = option_2(S = spot_price, K = strike_price, r = risk_free, 
                      today = spot_date, T = strike_date, 
                      vol = vol, dividend = div_yield)

# use the price method to compute the price of the option
put, call = google_example.price()

print('{} current option prices are: \n'.format(tk.info['longName']))
print('Put price: {:.3f} \nCall price: {:.3f}'.format(put, call))

Alphabet Inc. current option prices are: 

Put price: 5.396 
Call price: 4.962


Since the information downloaded is not in real time, the results are not fully reliable. However, it would be possible to get more accurate option prices, if we were able to obtain the needed values from a faster data provider.

# Adding Option Greeks
The Option Greeks are a set of risk measures that indicate how sensitive an option's price is to various factors, such as changes in the price of the underlying asset, time decay, volatility, and interest rates. Therefore, they can help understanding the position that the company could have if it had a new option to the portfolio.

## Delta
Delta measures the sensitivity of the option's price to change with respect to changes in the price of the underlying asset. It represents the change in the option's price for a $1 move in the price of the underlying asset:
- For a put option, its value ranges from -1 to 0: for example, a Delta of -0.5 means that if the underlying asset's price increases by $1, the option price will decrease by $0.5.
- For a call option, its value ranges from 0 to 1: for example, a Delta of 0.5 means that if the underlying asset's price increases by $1, the option's price will increase by $0.50.

Delta can be used to hedge the portfolio position by ensuring that the overall portfolio has a net delta close to zero, making it less sensitive to small changes in the underlying asset's price.

</br>
</br>

<div style="text-align: center">  


$\Delta = \frac{\partial V}{\partial S}$

$\Delta_{call} = \Phi(d1)$

$\Delta_{put} = -\Phi(-d1)$

</div>

## Theta
Theta measures the sensitivity of the option's price to the passage of time, also known as time decay. It represents the amount by which the option's price decreases as time to expiration decreases by one day. Near expiration Theta tends to increase (in absolute value) as the option approaches expiration, especially for at-the-money options, since a slight increase or decrease could mean a bigger win or loss. </br>
It can also happen that Theta is negative, indicating that the option's value decreases over time, which is common for both call and put options.

</br>
</br>
<div style="text-align: center">  

$\Theta = -\frac{\partial V}{\partial \tau}$

$\Theta_{call} = -\frac{S\phi(d1)\sigma}{2\tau} - rK\exp{(-rT)}\Phi(d2)$

$\Theta_{put} = -\frac{S\phi(d1)\sigma}{2\tau} + rK\exp{(-rT)}\Phi(-d2)$

</div>

## Gamma
Gamma measures the rate of change of delta with respect to changes in the underlying asset's price. It indicates how much the delta will change if the underlying asset's price changes by $1.
- High Gamma: Indicates that delta is highly sensitive to changes in the underlying asset's price, which is typical for at-the-money options.
- Low Gamma: Indicates that delta is less sensitive to changes in the underlying asset's price, typical for deep in-the-money or out-of-the-money options.

Gamma is monitored to understand how delta might change with large price movements, helping to manage the risk of the delta-hedged portfolios.

</br>
</br>
<div style="text-align: center">  

$\Gamma = \frac{\partial \Delta}{\partial S} = \frac{\partial^2 V}{\partial S^2}$

$\Gamma = \frac{\phi(d1)}{S\sigma\sqrt{\tau}}$

</div>

## Vega
Vega measures the sensitivity of the option's price to changes in the implied volatility of the underlying asset. It represents the change in the option's price for a 1% change in implied volatility.
- High Vega: Indicates that the option's price is highly sensitive to changes in volatility, common for longer-dated options or options that are at-the-money.
- Low Vega: Indicates that the option's price is less sensitive to changes in volatility.

</br>
</br>
<div style="text-align: center">  

$\upsilon = \frac{\partial V}{\partial \sigma}$

$\upsilon = S\phi(d1)\sqrt{\tau}$

</div>

## Rho
Rho measures the sensitivity of the option's price to changes in the risk-free interest rate. It represents the change in the option's price for a 1% change in the risk-free interest rate.
- Call Option: A positive rho means that the option's price increases as interest rates rise.
- Put Option: A negative rho means that the option's price decreases as interest rates rise.


</br>
</br>
<div style="text-align: center">  

$\rho = \frac{\partial V}{\partial r}$

$\rho_{call} = K\tau\exp{(-rT)}\Phi(d2)$

$\rho_{put} = -K\tau\exp{(-rT)}\Phi(-d2)$

</div>

In [10]:
class greeks(option_2):
    
    # compute delta
    def delta(self):
        d1_put = self.d1_put()
        d1_call = self.d1_call()
        
        delta_put = norm.cdf(d1_put) -1
        delta_call = norm.cdf(d1_call)
        
        return delta_put, delta_call
    
    # compute theta
    def theta(self):
        d1_put = self.d1_put()
        d1_call = self.d1_call()
        d2_put = self.d2_put()
        d2_call = self.d2_call()

        theta_call = - S * norm.pdf(d1_call) * self.vol_call / (2 * sqrt(self.t)) - self.r * self.K * e**(-self.r * self.t) * norm.cdf(d2_call)
        theta_put = - S * norm.pdf(d1_put) * self.vol_put / (2 * sqrt(self.t)) + self.r * self.K * e**(-self.r * self.t) * norm.cdf(-d2_put)
    
        # The division /365 shows the effect of decay day-by-day
        return theta_put/365, theta_call/365
    
    # compute gamma
    def gamma(self):
        d1_put = self.d1_put()
        d1_call = self.d1_call()

        gamma_put = norm.pdf(d1_put)/(self.S * self.vol_call * sqrt(self.t))
        gamma_call = norm.pdf(d1_call)/(self.S * self.vol_call * sqrt(self.t))

        return gamma_put, gamma_call

    # compute vega
    def vega(self):
        d1_put = self.d1_put()
        d1_call = self.d1_call()

        vega_put = self.S * norm.pdf(d1_put) * sqrt(self.t)
        vega_call = self.S * norm.pdf(d1_call) * sqrt(self.t)

        # It is more meaningful to see my sensitivity to a 1% change in volatility, so I multiply by 0.01
        return vega_put * 0.01, vega_call * 0.01

    # compute rho
    def rho(self):
        d2_put = self.d2_put()
        d2_call = self.d2_call()

        rho_put = -self.K * self.t * e**(- self.r * self.t) * norm.cdf(-d2_put,)
        rho_call = self.K * self.t * e**(- self.r * self.t) * norm.cdf(d2_call)

        # It is more meaningful to see my sensitivity to a 1% change in interest rates, so I multiply by 0.01
        return rho_put * 0.01, rho_call * 0.01



S = 42
K = 40
r = 0.1
std = 0.2

option_example = greeks(S = S, K = K, r = r, 
                      today = datetime(2020, 1, 1).date(), T = datetime(2020,7, 1).date(), 
                      vol = [std, std], dividend = 0)

put, call = option_example.price()

print('Put price: {:.3f}, Call price: {:.3f}'.format(put, call))

delta_put, delta_call = option_example.delta()
theta_put, theta_call = option_example.theta()
gamma_put, gamma_call = option_example.gamma()
vega_put, vega_call = option_example.vega()
rho_put, rho_call = option_example.rho()

print('Delta Call: {:.4f}'.format(delta_call))
print('Theta Call: {:.4f}'.format(theta_call))
print('Gamma Call: {:.4f}'.format(gamma_call))
print('Vega Call: {:.4f}'.format(vega_call))
print('Rho Call: {:.4f}'.format(rho_call))

Put price: 0.808, Call price: 4.753
Delta Call: 0.7791
Theta Call: -0.0125
Gamma Call: 0.0500
Vega Call: 0.0880
Rho Call: 0.1395


# Greeks in Practice
Now I set a pratical example using the Google stock, and the data we extracted with the script above:

In [11]:
google_example = greeks(S = spot_price, K = strike_price, r = risk_free, 
                      today = spot_date, T = strike_date, 
                      vol = vol, dividend = div_yield)

delta_put, delta_call = google_example.delta()
theta_put, theta_call = google_example.theta()
gamma_put, gamma_call = google_example.gamma()
vega_put, vega_call = google_example.vega()
rho_put, rho_call = google_example.rho()

print('{} current option greeks are: \n'.format(tk.info['longName']))
print('Delta Put: {:.4f}, Delta Call: {:.4f}'.format(delta_put, delta_call))
print('Theta Put: {:.4f}, Theta Call: {:.4f}'.format(theta_put, theta_call))
print('Gamma Put: {:.4f}, Gamma Call: {:.4f}'.format(gamma_put, gamma_call))
print('Vega Put: {:.4f}, Vega Call: {:.4f}'.format(vega_put, vega_call))
print('Rho Put: {:.4f}, Rho Call: {:.4f}'.format(rho_put, rho_call))

Alphabet Inc. current option greeks are: 

Delta Put: -0.4976, Delta Call: 0.5052
Theta Put: -0.0114, Theta Call: -0.0352
Gamma Put: 0.0280, Gamma Call: 0.0280
Vega Put: 0.1844, Vega Call: 0.1844
Rho Put: -0.0691, Rho Call: 0.0615


# **References**

## **Textbook**

- **[Options, Futures, and Other Derivatives, 11th Edition](https://elibrary.pearson.de/book/99.150005/9781292410623)**


<br/>
<br/>
<br/>
<b>Disclaimer: </b><br>
The code written in this file is for educational and informational purposes only. It is not intended as financial advice, and I make no guarantee of its accuracy, completeness, or suitability for any specific purpose. Any use of this code is at your own risk. I am not liable for any damages or losses arising from its use. Always perform your own testing and consult a financial professional before making any trading or investment decisions.